## 处理异常
在网络不好的情况下，如果出现了异常，该怎么办呢？这时如果不处理这些异常，程序很可能因报错而终止运行，所以异常处理还是十分有必要的。  
urllib 的 error 模块定义了由 request 模块产生的异常。如果出现了问题，request 模块便会抛出 error 模块中定义的异常。

### URLError
URLError 类来自 urllib 库的 error 模块，它继承自 OSError 类，是 error 异常模块的基类，由 request 模块生的异常都可以通过捕获这个类来处理。  
它具有一个属性 reason，即返回错误的原因。

In [1]:
from urllib import request, error
try:
    response = request.urlopen('http://cuiqingcai.com/index.htm')
except error.URLError as e:
    print(e.reason)

Not Found


### HTTPError
它是 URLError 的子类，专门用来处理 HTTP 请求错误，比如认证请求失败等。它有如下 3 个属性。  
- code：返回 HTTP 状态码，比如 404 表示网页不存在，500 表示服务器内部错误等。
- reason：同父类一样，用于返回错误的原因。
- headers：返回请求头。

In [2]:
from urllib import request,error
try:
    response = request.urlopen('http://cuiqingcai.com/index.htm')
except error.HTTPError as e:
    print(e.reason, e.code, e.headers, sep='\n')

Not Found
404
Server: GitHub.com
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
ETag: "5f73aee9-247b"
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; img-src data:; connect-src 'self'
x-proxy-cache: MISS
X-GitHub-Request-Id: E7C4:128F:B73618:CC2A1F:61EE22F2
Accept-Ranges: bytes
Date: Mon, 24 Jan 2022 03:56:00 GMT
Via: 1.1 varnish
Age: 94
X-Served-By: cache-hkg17922-HKG
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1642996560.330315,VS0,VE1
Vary: Accept-Encoding
X-Fastly-Request-ID: afb8a31e1f31b5664007e9e90e0ef49f899443e0
X-Cache-Lookup: Cache Miss
X-Cache-Lookup: Cache Miss
X-Cache-Lookup: Cache Miss
Content-Length: 9339
X-NWS-LOG-UUID: 4187327800251572993
Connection: close
X-Cache-Lookup: Cache Miss




因为 URLError 是 HTTPError 的父类，所以可以先选择捕获子类的错误，再去捕获父类的错误，所以上述代码更好的写法如下：

In [3]:
from urllib import request, error

try:
    response = request.urlopen('http://cuiqingcai.com/index.htm')
except error.HTTPError as e:
    print(e.reason, e.code, e.headers, sep='\n')
except error.URLError as e:
    print(e.reason)
else:
    print('Request Successfully')

Not Found
404
Server: GitHub.com
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
ETag: "5f73aee9-247b"
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; img-src data:; connect-src 'self'
x-proxy-cache: MISS
X-GitHub-Request-Id: E7C4:128F:B73618:CC2A1F:61EE22F2
Accept-Ranges: bytes
Date: Mon, 24 Jan 2022 03:56:40 GMT
Via: 1.1 varnish
Age: 134
X-Served-By: cache-hkg17935-HKG
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1642996600.497094,VS0,VE1
Vary: Accept-Encoding
X-Fastly-Request-ID: 5e30e1bf27d1446d8e22865c70df8feaec9d9917
X-Cache-Lookup: Cache Miss
X-Cache-Lookup: Cache Miss
X-Cache-Lookup: Cache Miss
Content-Length: 9339
X-NWS-LOG-UUID: 18274133567145659667
Connection: close
X-Cache-Lookup: Cache Miss




这样就可以做到先捕获 HTTPError，获取它的错误状态码、原因、headers 等信息。如果不是 HTTPError 异常，就会捕获 URLError 异常，输出错误原因。最后，用 else 来处理正常的逻辑。这是一个较好的异常处理写法。  
有时候，reason 属性返回的不一定是字符串，也可能是一个对象。再看下面的实例：

In [6]:
import socket
import urllib.request
import urllib.error

try:
    response = urllib.request.urlopen('https://www.baidu.com', timeout=0.01)
except urllib.error.URLError as e:
    print(type(e.reason))
    if isinstance(e.reason, socket.timeout):
        print('TIME OUT')

<class 'socket.timeout'>
TIME OUT


## 解析链接
urllib 库里还提供了 parse 这个模块，它定义了处理 URL 的标准接口，例如实现 URL 各部分的抽取、合并以及链接转换。它支持如下协议的 URL 处理：file、ftp、gopher、hdl、http、https、imap、mailto、 mms、news、nntp、prospero、rsync、rtsp、rtspu、sftp、 sip、sips、snews、svn、svn+ssh、telnet 和 wais。

### urlparse()
该方法可以实现URL的识别和分段

In [9]:
from urllib.parse import urlparse

response = urlparse('http://www.baidu.com/index.html;user?id=5#comment')
print(type(response),'\n',response)

<class 'urllib.parse.ParseResult'> 
 ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html', params='user', query='id=5', fragment='comment')


返回结果是一个 ParseResult 类型的对象，它包含 6 部分，分别是 scheme、netloc、path、params、query 和 fragment。  
`http://www.baidu.com/index.html;user?id=5#comment`   
标准的链接格式，具体如下：  
`<scheme>://<netloc>/<path>;<params>?<query>#<fragment>`  
#### urlparse API  
urlparse(url, scheme='', allow_fragments=True)
- url：这是必填项，即待解析的 URL
- scheme：它是默认的协议（比如 http 或 https 等）。假如这个链接没有带协议信息，会将这个作为默认的协议。scheme 参数只有在 URL 中不包含 scheme 信息时才生效。如果 URL 中有 scheme 信息，就会返回解析出的 scheme。
- allow_fragments：即是否忽略 fragment。如果它被设置为 False，fragment 部分就会被忽略，它会被解析为 path、parameters 或者 query 的一部分，而 fragment 部分为空。

In [11]:
help(urlparse)

Help on function urlparse in module urllib.parse:

urlparse(url, scheme='', allow_fragments=True)
    Parse a URL into 6 components:
    <scheme>://<netloc>/<path>;<params>?<query>#<fragment>
    Return a 6-tuple: (scheme, netloc, path, params, query, fragment).
    Note that we don't break the components up in smaller bits
    (e.g. netloc is a single string) and we don't expand % escapes.



返回结果 ParseResult 实际上是一个元组，我们可以用索引顺序来获取，也可以用属性名获取。

In [18]:
from urllib.parse import urlparse

response = urlparse('http://www.baidu.com/index.html;user?id=5#comment')
print(response[0],response[1],response.path,response.params,response.query,response.fragment,sep='\n')

http
www.baidu.com
/index.html
user
id=5
comment


#### urlunparse
实现 URL 的构造,将链接各个部分组合成完整链接的方法.它接受的参数是一个可迭代对象，但是它的长度必须是 6，否则会抛出参数数量不足或者过多的问题。

In [21]:
from urllib.parse import urlunparse

# data = ['http','www.baidu.com','index.html','student','id=2','comment']
data = ('http','www.baidu.com','index.html','student','id=2','comment')
print(urlunparse(data))

http://www.baidu.com/index.html;student?id=2#comment


#### urlsplit
作用和urlparse类似，只不过不再单独解析param选项，而是将param并入path

In [23]:
from urllib.parse import urlsplit

response = urlsplit("http://www.baidu.com/index.html;student?id=2#age")
print(response)
print(response[0],response[1])

SplitResult(scheme='http', netloc='www.baidu.com', path='/index.html;student', query='id=2', fragment='age')
http www.baidu.com


#### urlunsplit
作用和`urlunparse`类似，它接受的参数是一个可迭代对象，但是它的长度必须是 5，否则会抛出参数数量不足或者过多的问题。省去了`param`

In [27]:
from urllib.parse import urlunsplit

data = ['http','www.alibaba.com','index.html;user','id=2','age']
print(urlunsplit(data))

http://www.alibaba.com/index.html;user?id=2#age


#### urljoin
提供一个 base_url（基础链接）作为第一个参数，将新的链接作为第二个参数，该方法会分析 base_url 的 scheme、netloc 和 path 这 3 个内容并**对新链接缺失的部分进行补充，最后返回结果。**  
base_url 提供了三项内容 scheme、netloc 和 path。如果**这 3 项在新的链接里不存在**，就对新链接予以补充；如果新的链接存在这三项，就使用新的链接的部分。而 base_url 中的 params、query 和 fragment 是不起作用的。

In [30]:
from urllib.parse import urljoin

print(urljoin('http://www.baidu.com', 'FAQ.html'))
print(urljoin('http://www.baidu.com', 'https://cuiqingcai.com/FAQ.html'))
print(urljoin('http://www.baidu.com/about.html', 'https://cuiqingcai.com/FAQ.html'))
print(urljoin('http://www.baidu.com/about.html', 'https://cuiqingcai.com/FAQ.html?question=2'))
print(urljoin('http://www.baidu.com?wd=abc', 'https://cuiqingcai.com/index.php'))
print(urljoin('http://www.baidu.com', '?category=2#comment'))
print(urljoin('www.baidu.com', '?category=2#comment'))
print(urljoin('www.baidu.com#comment', '?category=2')) #只补充三项内容，#comment不属于scheme，netloc，path

http://www.baidu.com/FAQ.html
https://cuiqingcai.com/FAQ.html
https://cuiqingcai.com/FAQ.html
https://cuiqingcai.com/FAQ.html?question=2
https://cuiqingcai.com/index.php
http://www.baidu.com?category=2#comment
www.baidu.com?category=2#comment
www.baidu.com?category=2


#### urlencode
它在构造 GET 请求参数的时候非常有用，先声明了一个字典来将参数表示出来，然后调用 urlencode() 方法将其**序列化**为 GET 请求参数。


In [35]:
from urllib.parse import urlencode

data = {
    'name':'lgr',
    'age':'12'
}
baseurl = "http://www.baidu.com?" 
url = baseurl + urlencode(data)
print(url)

http://www.baidu.com?name=lgr&age=12


#### parse_qs
有了序列化，必然就有反序列化。如果我们有一串 GET 请求参数，利用 parse_qs() 方法，就可以将它转回字典

In [37]:
from urllib.parse import parse_qs

param = 'name=lgr&age=12'
print(parse_qs(param))

{'name': ['lgr'], 'age': ['12']}


#### parse_qsl
将参数转化为元组组成的列表

In [38]:
from urllib.parse import parse_qsl

param = 'name=lgr&age=12'
print(parse_qsl(param))

[('name', 'lgr'), ('age', '12')]


#### quote
将内容转化为 URL 编码的格式。URL 中带有中文参数时，有时可能会导致乱码的问题，此时用这个方法可以将中文字符转化为 URL 编码
#### unquote

In [40]:
from urllib.parse import quote,unquote

keyword = '壁纸'
url = 'https://www.baidu.com/s?wd=' + quote(keyword)
print(url)
print(unquote(url))

https://www.baidu.com/s?wd=%E5%A3%81%E7%BA%B8
https://www.baidu.com/s?wd=壁纸
